In [1]:
import os, time
import yandexdriver_update
from datetime import date, timedelta, datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys

yandexdriver_update.yandexdriver_update()

url="http://monitoring.mis66.ru/"
url2="http://10.3.224.196"
username=os.getenv('MDMU_USERNAME_1')
password=os.getenv('MDMU_PASSWORD_1')
today = date.today().strftime("%d.%m.%Y")
yesterday = (date.today() - timedelta(days=1)).strftime("%d.%m.%Y")
reports_to_check_and_send=['100.1']# <-----------------------------------\
reports_to_duplicate_and_send=['1159','1259','1157','1166','1197']# <------сюдой
reports_to_duplicate=['1296']# <-----------------------------------------/
#canceled by MZSO at 21.04.2025 # 03-01-82/9001
canceled_reports=['100.1',
                  '209','270',
                  '325', '381', '382', '383', '384', 
                  '410', '480', '440', '492', '494',
                  '566', '636', '665', '667', '741', '748', 
                  '850', '868', '989', '997', 
                  '1037', '1046', '1059', '1063', '1064','1092',
                  '1102','1122','1123','1124','1129','1130','1131','1132','1134', '1137', '1159', '1160', '1166', '1169', '1174', '1195', '1196', 
                  '1201', '1225', '1228','1230','1236','1237', '1257', '1258', '1259', '1261', '1262','1275',  '1276', '1296',
                  '1303','1304','1321','1332','1339','1341','1356','1357', '1359', '1364', '1368', '1369', '1370', '1371', '1372', '1373', '1374', '1375', '1376', '1377', '1378','1379','1380', '1382','1396',
                  '1411', '1413', '1416', '1417', '1418', '1422','1423', '1459', '1460', '1461', '1462', 
                  '1501', '1502', '1514', '1526', '1527', '1542', '1547', 
                  '1624','1691','1692', '2902']
#reports_for_laba=['1169','1321','1276','1228','1201']# <----------------\
#reports_for_rao=['854','1296']# <-----------------------------------------я не помню что это

def click_id(id):
    driver.find_element(By.ID, id).click()
    time.sleep(1)
    
def click_xpath(xpath):
    if len(driver.find_elements(By.XPATH, xpath)):
        driver.find_element(By.XPATH, xpath).click()
        return True
        time.sleep(1)
    return False

def type_xpath(xpath,string):
    driver.find_element(By.XPATH, xpath).clear()
    driver.find_element(By.XPATH, xpath).send_keys(string)
    time.sleep(1)

def login():
    type_xpath("//*[@id='dijit_form_ValidationTextBox_0']",username)
    type_xpath("//*[@id='dijit_form_ValidationTextBox_1']",password)
    click_xpath("//*[text()=' ОК ']")
    print('Login sequence completed')
    time.sleep(10)
    
def logout():
    click_xpath("//*[@class='dijitReset dijitInline dijitIcon balanceApplicationTitleCloseButton']")
    driver.quit()
    print('Logout sequence completed')
    time.sleep(10)

def filter_all_from_yesterday():
    click_id("dijit_form_ComboButton_3_label")
    click_xpath("//td[.=' Отобрать... ']")
    time.sleep(1)
    inputs=driver.find_elements(By.XPATH, "//div[@class='dijitTabContainerTopChildWrapper dijitVisible']//input[@class='dijitReset dijitInputInner']")
    for i in inputs:
        i.clear()
    checkboxes = driver.find_elements(By.XPATH, "//*[text()='Первичные отчеты: Отбор']/../..//input[@type='checkbox']")
    for checkbox in checkboxes:
        if not checkbox.is_selected():
            checkbox.click()
    time.sleep(1)
    type_xpath("//*[contains(text(),'Дата с:')]/../..//input[@class='dijitReset dijitInputInner'][1]",yesterday)
    time.sleep(1)
    
    click_xpath("//span[.=' OK ']")
    print('Filter all from yesterday sequence completed')

def show_all():
    click_id("dijit_form_ComboButton_2_label")
    click_xpath("//td[.=' Все ']")
    time.sleep(5)
    print('Show all sequence completed')
    
def send_report(report):
    click_xpath("//td[text()='"+today+"']/..//td[text()='"+report+"']")
    click_xpath("//span[@class='dijitReset dijitInline dijitIcon PCRightCustomIconBig']")
    click_xpath("//*[text()=' Да ']")
    print('Send report sequence completed',report)
    time.sleep(5)

def check_report(report):
    click_xpath("//td[text()='"+today+"']/..//td[text()='"+report+"']")
    click_xpath("(//*[@class='dijitReset dijitInline dijitIcon PCCheckCustomIconBig'])[1]")
    checkboxes = driver.find_elements(By.XPATH, "//*[text()='Первичные отчеты: Параметры проверки']/../..//input[@type='checkbox']")
    for checkbox in checkboxes:
        if checkbox.is_selected():
            checkbox.click()
    time.sleep(1)
    click_xpath("//b[text()=' OK ']")
    time.sleep(10)
    click_xpath("//b[text()=' Закрыть ']")
    print('Check report sequence completed',report)
    time.sleep(5)
    
def delete_report(report):
    if click_xpath("//td[text()='"+today+"']/..//td[text()='"+report+"']"):
        click_xpath("(//*[@class='dijitReset dijitInline dijitIcon PCreduceCustomIconBig'])[3]")
        click_xpath("//*[text()=' Да ']")
        print('Delete report sequence completed',report)
        time.sleep(5)
        return True
    print('Report for delete not found',report)
    return False

def duplicate_report_from_yesterday(report):
    click_xpath("//td[text()='"+yesterday+"']/..//td[text()='"+report+"']")
    click_xpath("(//*[@class='dijitReset dijitInline dijitIcon PCcopyCustomIconBig'])[1]")
    type_xpath("//*[contains(text(),'Дата:')]/../..//input[@class='dijitReset dijitInputInner']",today)
    click_xpath("//b[text()=' OK ']")
    print('Duplicate report sequence completed',report)
    time.sleep(5)
    
def delete_duplicate_and_send_report(report):
    delete_report(report)
    duplicate_report_from_yesterday(report)
    send_report(report)

def filter_report(report):
    click_id("dijit_form_ComboButton_3_label")
    click_xpath("//td[.=' Отобрать... ']")
    time.sleep(1)
    inputs=driver.find_elements(By.XPATH, "//div[@class='dijitTabContainerTopChildWrapper dijitVisible']//input[@class='dijitReset dijitInputInner']")
    for index,input in enumerate(inputs):
        print(index, input)
        input.clear()
        if index == 0:
            input.send_keys(report)
    checkboxes = driver.find_elements(By.XPATH, "//*[text()='Первичные отчеты: Отбор']/../..//input[@type='checkbox']")
    for checkbox in checkboxes:
        if not checkbox.is_selected():
            checkbox.click()
    time.sleep(1)
    type_xpath("//*[contains(text(),'Дата с:')]/../..//input[@class='dijitReset dijitInputInner'][1]",yesterday)
    time.sleep(1)
    
    click_xpath("//span[.=' OK ']")
    print('Filter', report, 'sequence completed')

def exit_monitoring():
    click_xpath("//*[@class='dijitReset dijitInline dijitIcon balanceApplicationTitleCloseButton']")

def open_report(report):
    if click_xpath("//td[text()='"+today+"']/..//td[text()='"+report+"']"):
        click_xpath("(//*[@class='dijitReset dijitInline dijitIcon PCapplicationsCustomIconBig'])[1]")
        click_xpath("//*[text()=' Да ']")
        print('Open report sequence completed',report)
        time.sleep(5)
        return True
    print('Report for opening not found',report)
    return False

def send_report_1743():
    filter_report('1743')
    open_report('1743')
    click_xpath("//*[@ids='R10C2']")
    time.sleep(1)
    webdriver.ActionChains(driver).send_keys(today).perform()
    time.sleep(1)
    webdriver.ActionChains(driver).send_keys(Keys.ENTER).perform()
    time.sleep(1)
    click_xpath('//*[@class="dijitReset dijitInline dijitIcon balanceApplicationTitleCloseButton"]')
    time.sleep(1)
    click_xpath("//*[text()=' Да ']")
    time.sleep(5)
    send_report('1743')

print(datetime.now())

current dir: C:\Users\User\Anaconda\Github
Current version: v25.10.0-stable
Latest version: v25.10.0-stable
Already at the latest version, skipping download
2025-11-01 11:30:29.252039


In [6]:
#Initialization
binary_yandex_driver_file = 'yandexdriver.exe'
# Create the profile directory if it doesn't exist:
profile_directory = os.path.join(os.getcwd(), "YandexUserMonitoring")
os.makedirs(profile_directory, exist_ok=True)
options = webdriver.ChromeOptions()

#hide automation from remote site:
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features=AutomationControlled")

# using dedicated profile dir:
options.add_argument(f"--user-data-dir={profile_directory}")

driver = webdriver.Chrome(options=options, service=Service(executable_path=binary_yandex_driver_file))
driver.set_page_load_timeout(10)

for x in range(100):
    try:
        driver.get(url)
        break
    except TimeoutException:
        print("url:", url, "invalid", 'trying second url')
        try:
            driver.get(url2)
            break
        except TimeoutException:
            print("url2:", url2, "invalid", 'trying first url')
driver.implicitly_wait(1)
print(driver.title)

login()

# today = date.today().strftime("%d.%m.%Y")
today="10.08.2025"
yesterday = (date.today() - timedelta(days=1)).strftime("%d.%m.%Y")
# yesterday="19.07.2024"
filter_all_from_yesterday()
show_all()

SessionNotCreatedException: Message: session not created: probably user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
	GetHandleVerifier [0x0x7ff70f8412e5+24661]
	(No symbol) [0x0x7ff70f7827e0]
	(No symbol) [0x0x7ff70f5d15fa]
	(No symbol) [0x0x7ff70f60fe2d]
	(No symbol) [0x0x7ff70f60a44b]
	(No symbol) [0x0x7ff70f660cbf]
	(No symbol) [0x0x7ff70f660420]
	(No symbol) [0x0x7ff70f6539d3]
	(No symbol) [0x0x7ff70f61936f]
	(No symbol) [0x0x7ff70f61a713]
	GetHandleVerifier [0x0x7ff70fbf324d+3899325]
	GetHandleVerifier [0x0x7ff70fc20662+4084690]
	GetHandleVerifier [0x0x7ff70f917bf9+903529]
	GetHandleVerifier [0x0x7ff70f91f3ce+934206]
	(No symbol) [0x0x7ff70f78c2eb]
	(No symbol) [0x0x7ff70f787ea4]
	(No symbol) [0x0x7ff70f788057]
	(No symbol) [0x0x7ff70f778fa5]
	Bas

In [4]:
for report in reports_to_check_and_send:
    if report in canceled_reports:
        print(f'report {report} is canceled')
        continue
    else:
        print(f'sending report {report}')
        
    filter_report(report)
    check_report(report)
    send_report(report)
    
for report in reports_to_duplicate_and_send:
    if report in canceled_reports:
        print(f'report {report} is canceled')
        continue
    else:
        print(f'sending report {report}')
    filter_report(report)
    delete_report(report)
    duplicate_report_from_yesterday(report)
    send_report(report)

for report in reports_to_duplicate:
    if report in canceled_reports:
        print(f'report {report} is canceled')
        continue
    else:
        print(f'sending report {report}')
    filter_report(report)
    delete_report(report)
    duplicate_report_from_yesterday(report)


report 100.1 is canceled
report 1159 is canceled
report 1259 is canceled
sending report 1157
0 <selenium.webdriver.remote.webelement.WebElement (session="7b7a938338f5f6452b0c3fe001db0ab8", element="f.5F535215061590018FC0B7C9E0288F97.d.DB7849F45497EDDC5940F4466EFA558C.e.225")>
1 <selenium.webdriver.remote.webelement.WebElement (session="7b7a938338f5f6452b0c3fe001db0ab8", element="f.5F535215061590018FC0B7C9E0288F97.d.DB7849F45497EDDC5940F4466EFA558C.e.268")>
2 <selenium.webdriver.remote.webelement.WebElement (session="7b7a938338f5f6452b0c3fe001db0ab8", element="f.5F535215061590018FC0B7C9E0288F97.d.DB7849F45497EDDC5940F4466EFA558C.e.270")>
3 <selenium.webdriver.remote.webelement.WebElement (session="7b7a938338f5f6452b0c3fe001db0ab8", element="f.5F535215061590018FC0B7C9E0288F97.d.DB7849F45497EDDC5940F4466EFA558C.e.226")>
4 <selenium.webdriver.remote.webelement.WebElement (session="7b7a938338f5f6452b0c3fe001db0ab8", element="f.5F535215061590018FC0B7C9E0288F97.d.DB7849F45497EDDC5940F4466EFA5

In [5]:
send_report_1743()
filter_report('854')
send_report('854')

0 <selenium.webdriver.remote.webelement.WebElement (session="7b7a938338f5f6452b0c3fe001db0ab8", element="f.5F535215061590018FC0B7C9E0288F97.d.DB7849F45497EDDC5940F4466EFA558C.e.755")>
1 <selenium.webdriver.remote.webelement.WebElement (session="7b7a938338f5f6452b0c3fe001db0ab8", element="f.5F535215061590018FC0B7C9E0288F97.d.DB7849F45497EDDC5940F4466EFA558C.e.798")>
2 <selenium.webdriver.remote.webelement.WebElement (session="7b7a938338f5f6452b0c3fe001db0ab8", element="f.5F535215061590018FC0B7C9E0288F97.d.DB7849F45497EDDC5940F4466EFA558C.e.800")>
3 <selenium.webdriver.remote.webelement.WebElement (session="7b7a938338f5f6452b0c3fe001db0ab8", element="f.5F535215061590018FC0B7C9E0288F97.d.DB7849F45497EDDC5940F4466EFA558C.e.756")>
4 <selenium.webdriver.remote.webelement.WebElement (session="7b7a938338f5f6452b0c3fe001db0ab8", element="f.5F535215061590018FC0B7C9E0288F97.d.DB7849F45497EDDC5940F4466EFA558C.e.803")>
Filter 1743 sequence completed
Open report sequence completed 1743
Send report s

In [ ]:
exit_monitoring()
driver.quit()
print('Mission complete')
print(datetime.now())
time.sleep(60*5)
sys.exit(0)

Mission complete
2025-08-10 13:19:04.219984


In [ ]:
driver.quit()
sys.exit(0)

In [ ]:
options = webdriver.ChromeOptions()
binary_yandex_driver_file = 'chromedriver.exe'
service = webdriver.chrome.service.Service(executable_path=binary_yandex_driver_file)
driver = webdriver.Chrome(service=service, options=options)
driver.set_page_load_timeout(30)

for x in range(100):
    try:
        driver.get(url)
        break
    except TimeoutException:
        print("url:", url, "invalid", 'trying second url')
        try:
            driver.get(url2)
            break
        except TimeoutException:
            print("url2:", url2, "invalid", 'trying first url')
driver.implicitly_wait(1)
print(driver.title)

login()

today="22.07.2024"
yesterday="21.07.2024"
filter_all_from_yesterday()
show_all()

for report in reports_to_check_and_send:
    filter_report(report)
    check_report(report)
    send_report(report)
    
for report in reports_to_duplicate_and_send:
    filter_report(report)
    delete_report(report)
    duplicate_report_from_yesterday(report)
    send_report(report)

for report in reports_to_duplicate:
    filter_report(report)
    delete_report(report)
    duplicate_report_from_yesterday(report)
exit_monitoring()

In [ ]:
options = webdriver.ChromeOptions()
binary_yandex_driver_file = 'chromedriver.exe'
service = webdriver.chrome.service.Service(executable_path=binary_yandex_driver_file)
driver = webdriver.Chrome(service=service, options=options)
driver.set_page_load_timeout(30)

for x in range(100):
    try:
        driver.get(url)
        break
    except TimeoutException:
        print("url:", url, "invalid", 'trying second url')
        try:
            driver.get(url2)
            break
        except TimeoutException:
            print("url2:", url2, "invalid", 'trying first url')
driver.implicitly_wait(1)
print(driver.title)

login()
filter_all_from_yesterday()
show_all()